<a href="https://colab.research.google.com/github/IsaacFayle-Waters/algorithms_etc/blob/main/Copy_of_a_star_package_delivery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import heapq
from copy import deepcopy

class A_star_drones:
  def __init__(self,problem_size: int = 5):

    self.delivery_number = self.problem_s = problem_size
    self.delivery_area = [[0 for _ in range(problem_size)] for _ in range(problem_size)]

    self.open_list = []
    self.closed_list = set()

    self.g_cost = {}
    self.came_from = {}

    self.building_positions = set()
    self.package_positions = set()

    self.drone_moves = [[0,-1],[0,1],[-1,0],[1,0]] # up, down, left, right

    self.full_path = []

  #Print board
  def display_board(self):
    outline = 2 * self.delivery_number + 2# + self.delivery_number
    print('_'*outline)
    for row in self.delivery_area: # Join partly from claude.ai
      cells = ' '.join('D' if cell == 3 else
                       '#' if cell == 1 else
                       'P' if cell == 2 else
                       'o' if cell == 4 else
                       'X' if cell == 5 else
                       '.' for cell in row)
      print(f'| {cells} |')
    print('‾'*outline)

  def plot_path(self):
    print(self.full_path)
    for row,col in self.full_path:
      if self.delivery_area[row][col] == 0: #'.'
        self.delivery_area[row][col] = 4
      elif self.delivery_area[row][col] == 2: #'P'
        self.delivery_area[row][col] = 5

  #Initialise drone positin at (0,0) for now, and add start node to heap
  def init_drone_position(self):
    self.delivery_area[0][0] = 3
    heapq.heappush(self.open_list, (0, (0,0)))  # (cost, node)
    self.g_cost = {(0,0): 0}
    self.came_from = {(0,0):None}

  #Add buildings to scenario
  def place_buildings(self):
    p_size = self.problem_s
    for cell in range(p_size):
      empty = True
      while empty:
        rand_r = np.random.randint(0,p_size)
        rand_c = np.random.randint(0,p_size)
        #print(f"Before : Row : {rand_r}, Col : {rand_c}")
        if self.delivery_area[rand_r][rand_c] == 0:
          self.delivery_area[rand_r][rand_c] = 1
          self.building_positions.add((rand_r,rand_c))
          empty = False

  #Add Packages to scenario
  def stage_packages(self):
    p_size = self.problem_s
    open_positions = {pos for priority, pos in self.open_list} # From Claude.ai
    #Packages
    for cell in range(p_size):
      not_staged = True
      while not_staged:
        rand_r = np.random.randint(0,p_size)
        rand_c = np.random.randint(0,p_size)
        if ((rand_r, rand_c) not in self.building_positions and
            (rand_r, rand_c) not in open_positions and
            (rand_r, rand_c) not in self.package_positions):
            self.delivery_area[rand_r][rand_c] = 2
            self.package_positions.add((rand_r, rand_c))
            not_staged = False

  def delivery_route(self):
    current_position = (0, 0)  # Drone starts here
    total_cost = 0  # Start with zero cost
    full_path = [(0, 0)]
    unvisited = self.package_positions.copy()

    while unvisited:
      nearest_package = None
      nearest_cost = np.inf
      nearest_path = None
      for package in unvisited:
        path,cost = self.a_star(current_position, package)
        if cost < nearest_cost:
          nearest_cost = cost
          nearest_package = package
          nearest_path = path

    # Check if any path was found
      if nearest_path is None:
          print("No path to remaining packages!")
          return False  # Indicate failure

      current_position = nearest_package
      self.full_path.extend(nearest_path[1:])
      total_cost += nearest_cost
      unvisited.remove(nearest_package)

    return True

  #Manhatten distance
  def calulate_distance(self, x0, y0, x1, y1):
    return np.abs(x0 - x1) + np.abs(y0 - y1)

  def get_neighbors(self,position):
    row, col = position
    neighbours = []
    for move_row,move_col in self.drone_moves:
      new_row = row + move_row
      new_col = col + move_col
      bound = self.problem_s
      if 0 <= new_row < bound and 0 <= new_col < bound and self.delivery_area[new_row][new_col] != 1:
        neighbours.append((new_row,new_col))
    #print('Get Neigh : ',neighbours)
    return neighbours

  def a_star(self,start,goal):
    # Reset for this search
    self.open_list = []
    self.closed_list = set()
    self.g_cost = {start: 0}
    self.came_from = {start: None}

    # Initialize heap with start
    heapq.heappush(self.open_list, (0, start))
    while self.open_list:
      current_cost_f, current_position = heapq.heappop(self.open_list)
      #print(current_position)

      if current_position == goal: #Claude.ai
      # Reconstruct the path
        #print('eval')
        path = []
        node = goal
        while node is not None:
            path.append(node)
            node = self.came_from[node]
        path.reverse()  # Start to goal

        return path, self.g_cost[goal]

      self.get_neighbors(current_position)

      #Been evaluated
      if current_position in self.closed_list:
        continue
      #Now evaluating
      self.closed_list.add(current_position)

      neighbours = self.get_neighbors(current_position)
      #search possible positions
      for neighbor in neighbours:
        tentative_g = self.g_cost[current_position] + 1
        # check if better path
        if neighbor not in self.g_cost or tentative_g < self.g_cost[neighbor]: #claude.ai
          self.g_cost[neighbor] = tentative_g
          self.came_from[neighbor] = current_position

          h = self.calulate_distance(neighbor[0],neighbor[1],goal[0],goal[1])
          f = tentative_g + h
          heapq.heappush(self.open_list,(f, neighbor))

    return None, np.inf

if __name__ == "__main__":
  # Define problem - example problem size = 10
  # = 10 packages, 10 buildings, 10*10 area
  drone_prob = A_star_drones(problem_size=20)
  drone_prob.init_drone_position()
  drone_prob.place_buildings()
  drone_prob.stage_packages()
  #Display intitial state
  drone_prob.display_board()
  #A* search for all packages
  success = drone_prob.delivery_route()
  if success:
      drone_prob.plot_path()
      drone_prob.display_board()
  else:
      print("Route blocked - regenerating...")


__________________________________________
| D . . . . # # . . P . P P . . P . . . . |
| . . . . . . . . . . . . . . . . . . P . |
| . . . . . . . . . . . . . . . . . . . . |
| # . . . . . . . . . . . . P . . . # . . |
| # . . . . . . . . . P . . . . . . . . . |
| . . P . . P . . . # . . . . . . . . . . |
| . . . . P . . . . # # . . . . . . . . . |
| . . . . . . . . . . . . . . . . . . . . |
| . . . . . . . . . . . . . . # . . . P . |
| . . . . . P . . . . # . . . . # . P . . |
| . . . . . . . . . . . . . . . . . . . . |
| . . . . . . . . . . # . . . . . . . # . |
| . . . . . . . . . . . . . . . . P . . . |
| . . # . . . . . # . . . . . P . . . # # |
| . . . . . . . . . . . . . . . . . . . . |
| . . . . . . . . . # . . . . . . P . . . |
| . . . . . . . . P . # . . . . . . . . P |
| . . . . P . . . . . . . # . . . . . . . |
| . . . . . . . . . . P . . . . . . . . . |
| . . . . . . . . . . . . . . . . . . . . |
‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾
[(0, 1), (0, 2), (1, 2), (2, 2), (